<a href="https://colab.research.google.com/github/kekys778/Act-Probll-compLing-2024-Perfilev/blob/main/hw14/11/24Perfilev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from google.colab import files

In [153]:
! wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/notebooks/bow/labeled.csv.zip
!unzip -o labeled.csv.zip

df = pd.read_csv('labeled.csv')

--2024-11-22 16:39:53--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/notebooks/bow/labeled.csv.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/bow/labeled.csv.zip [following]
--2024-11-22 16:39:53--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/notebooks/bow/labeled.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1463222 (1.4M) [application/zip]
Saving to: ‘labeled.csv.zip.2’

labeled.csv.zip.2   100%[===================>]   1.39M  --.-KB/s    in 0.07s   

2024-11-22 16:39:53 (20.8

In [154]:
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


# ***Задание 1 (3 балла)***
У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)


# ***ОТВЕТ:***
Обе модели показывают сопоставимые результаты, но модель с дефолтной токенизацией немного лучше в плане общей точности и F1-score для класса 1.0. Это может быть связано с тем, что встроенная токенизация лучше адаптирована для работы с данными в текущем формате.

Использование razdel.tokenize в данном случае не даёт значительных преимуществ.

In [155]:
!pip install razdel

In [156]:
from razdel import tokenize

In [157]:
train, test = train_test_split(df, test_size=0.1, shuffle=True)

In [158]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [159]:
vectorizer1_base = CountVectorizer()
# в векторайзер нужно засовывать тексты строками (токенизация там встроена)
X = vectorizer1_base.fit_transform(train.comment)

In [160]:
vectorizer2_razdel = CountVectorizer(tokenizer = lambda x: [i.text for i in list(tokenize(x))])

X2 = vectorizer2_razdel.fit_transform(train.comment)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [161]:
x_test = vectorizer1_base.transform(test.comment)
x2_test = vectorizer2_razdel.transform(test.comment)

In [162]:
X.shape, x_test.shape

((12970, 63920), (1442, 63920))

In [163]:
X2.shape, x2_test.shape

((12970, 64820), (1442, 64820))

In [164]:
y = train.toxic.values
y_test = test.toxic.values

y2 = train.toxic.values
y2_test = test.toxic.values

In [165]:
clf = LogisticRegression(C=0.1, class_weight='balanced')

In [166]:
clf2 = LogisticRegression(C=0.1, class_weight='balanced')

In [167]:
clf.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [168]:
clf2.fit(X2, y2)

LogisticRegression(C=0.1, class_weight='balanced')

In [169]:
preds = clf.predict(x_test)

In [170]:
preds2 = clf2.predict(x2_test)

In [171]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.92      0.82      0.87       945
         1.0       0.72      0.86      0.78       497

    accuracy                           0.83      1442
   macro avg       0.82      0.84      0.82      1442
weighted avg       0.85      0.83      0.84      1442



In [172]:
print(classification_report(y2_test, preds2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.90      0.81      0.85       945
         1.0       0.70      0.83      0.76       497

    accuracy                           0.82      1442
   macro avg       0.80      0.82      0.81      1442
weighted avg       0.83      0.82      0.82      1442



# ***Задание 2 (3 балла)***
Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)
г) f1 мера каждого из классификаторов должна быть минимум 0.75

# ***ОТВЕТ***
Тексты действительно являются токсичными но из топ 10 совпадает только один текст, однако оба классификатора считают все полученные тексты токсичными

In [173]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [174]:
train, test = train_test_split(df, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [175]:
vectorizer1 = CountVectorizer(stop_words=stops, min_df=6, max_df=0.01, ngram_range=(1,10), analyzer = 'word')

In [176]:
vectorizer2 = TfidfVectorizer(stop_words=stops, min_df=6, max_df=0.3, analyzer='word', ngram_range=(1, 2), max_features=15000)

In [177]:
X = vectorizer1.fit_transform(train.comment)


In [178]:
X2 = vectorizer2.fit_transform(train.comment)

In [179]:
X_test = vectorizer1.transform(test.comment)


In [180]:
X2_test = vectorizer2.transform(test.comment)

In [181]:
y = train.toxic.values
y_test = test.toxic.values

y2 = train.toxic.values
y2_test = test.toxic.values

In [182]:
mnb = MultinomialNB(alpha=0.5)
mnb.fit(X, y)

preds = mnb.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89       951
         1.0       0.83      0.70      0.76       491

    accuracy                           0.85      1442
   macro avg       0.84      0.81      0.82      1442
weighted avg       0.85      0.85      0.84      1442



In [183]:
clf = KNeighborsClassifier(n_neighbors=5, metric='cosine')
clf.fit(X2, y2)
preds2 = clf.predict(X2_test)

print(classification_report(y2_test, preds2))

              precision    recall  f1-score   support

         0.0       0.83      0.78      0.80       951
         1.0       0.62      0.69      0.65       491

    accuracy                           0.75      1442
   macro avg       0.72      0.73      0.73      1442
weighted avg       0.76      0.75      0.75      1442



In [185]:
probs_mnb = mnb.predict_proba(X_test)[:, 1]
test['mnb_prob'] = probs_mnb
top_mnb = test.sort_values(by='mnb_prob', ascending=False).head(10)

In [186]:
probs_knn = clf.predict_proba(X2_test)[:, 1]
test['knn_prob'] = probs_knn
top_knn = test.sort_values(by='knn_prob', ascending=False).head(10)

In [187]:
top_mnb

,index,comment,toxic,mnb_prob
517,723,"Как известно, у Укр ины (т.е. окр ины), слепле...",1.0,1.0
432,1628,"именно так. смотрю на тебя, стаса, вестника ду...",1.0,1.0
1145,2251,Несправедливый раздел Русские себе почти всё з...,1.0,1.0
702,264,С каких пор порноскримеры нарушают что-то В то...,1.0,1.0
472,486,"та ну, хуйня это все про джентельменство . мен...",0.0,1.0
860,2354,Да Евген просто шлюшка без мнения - то блядь п...,1.0,1.0
641,14356,ДА КАКОГО ЕБАНОГО ХУЯ МНЕ ТЕПЕРЬ ЮТУБ РЕКОМЕНД...,1.0,1.0
352,12795,Да тупая баба. Видос недавно был: мразь какая ...,1.0,1.0
1381,2979,"Злобный дегенерат и педераст, здохший от перед...",1.0,1.0
821,6025,черт опущенный Гомикадзе Би опущенный гей на е...,1.0,1.0


In [188]:
top_knn

,index,comment,toxic,mnb_prob,knn_prob
400,13828,Маленький вонючий коммунистический пидорас-хуе...,1.0,0.999986,1.0
126,2106,"адекватную критику ряяяяяяяяяяяяяя, нипакниге,...",1.0,0.903411,1.0
1145,2251,Несправедливый раздел Русские себе почти всё з...,1.0,1.000000,1.0
902,4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,0.973838,1.0
272,9501,Позвольте угадаю СЗТ по которой ехал этот нищи...,1.0,0.823591,1.0
1310,14155,"Да, она просто всратая тян, с повадками быдла....",1.0,0.990509,1.0
901,14216,Ебать ты просто мировой заговор вскрыл. ВС !!!...,1.0,0.999972,1.0
778,6650,А грызуны?! Ебать у них зубища!\n,1.0,0.898928,1.0
127,14325,"Ну оль, ну вот че тебе надо? Иди в по сри, а э...",1.0,0.998335,1.0
284,7791,Среди умных людей -безусловно. Но стреляют то ...,1.0,0.868225,1.0


# *Задание 3 (4 балла - 1 балл за каждый классификатор)*
Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов.

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [189]:
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [190]:
!pip install pymorphy3



In [191]:
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def pymorphy_lemmatize(text):
    return [morph.parse(word)[0].normal_form for word in text.split()]

In [192]:
train, test = train_test_split(df, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [193]:
vectorizer = TfidfVectorizer(stop_words=stops, tokenizer = pymorphy_lemmatize, min_df=6, max_df=0.09, ngram_range=(1,3), analyzer = 'word')

In [194]:
X = vectorizer.fit_transform(train.comment)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['большой', 'весь', 'всё', 'ещё', 'мочь', 'нибыть', 'свой', 'хороший', 'это'] not in stop_words.
  warnings.warn(


In [195]:
X_test = vectorizer.transform(test.comment)

In [196]:
y = train.toxic.values
y_test = test.toxic.values

In [197]:
clf = LogisticRegression(C=0.1, class_weight='balanced')

In [198]:
clf.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [199]:
preds = clf.predict(X_test)

In [200]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.89      0.83      0.86       967
         1.0       0.70      0.79      0.74       475

    accuracy                           0.82      1442
   macro avg       0.79      0.81      0.80      1442
weighted avg       0.83      0.82      0.82      1442



In [201]:
import numpy as np
import pandas as pd

coefficients = clf.coef_

features = vectorizer.get_feature_names_out()

importance_df = pd.DataFrame({
    'feature': features,
    'importance': np.abs(coefficients[0])
})

importance_df = importance_df.sort_values(by='importance', ascending=False)


print(importance_df[['feature', 'importance']].head(20))


      feature  importance
5422    хохол    1.942252
4263  русский    1.686377
4963     твой    1.581726
5126    тупой    1.527846
990       год    1.473595
4331     свой    1.437361
3176    очень    1.406342
5096     тред    1.248485
5418   хохлов    1.192138
2662    нахуй    1.149717
5440      хуй    1.072001
404     блядь    1.064548
5462     цена    1.009129
758     время    0.988385
5445    хуйня    0.943154
1135    дебил    0.907322
971     говно    0.869293
1486     жопа    0.852371
4036   работа    0.840678
1029    город    0.827526


In [202]:
dtc = DecisionTreeClassifier(max_depth=5000, class_weight='balanced')

In [203]:
dtc.fit(X, y)

DecisionTreeClassifier(class_weight='balanced', max_depth=5000)

In [204]:
preds_dtc = dtc.predict(X_test)

In [205]:
print(classification_report(y_test, preds_dtc, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.82      0.77      0.79       967
         1.0       0.58      0.67      0.62       475

    accuracy                           0.73      1442
   macro avg       0.70      0.72      0.71      1442
weighted avg       0.75      0.73      0.74      1442



In [206]:

features_dtc = vectorizer.get_feature_names_out()

feat_importances = pd.DataFrame(
    dtc.feature_importances_,
    index=features_dtc,
    columns=["Importance"]
)

feat_importances.sort_values(by="Importance", ascending=False, inplace=True)

print(feat_importances.head(20))


         Importance
год        0.017132
хохол      0.015696
твой       0.013745
очень      0.013126
русский    0.010526
свой       0.010490
тупой      0.010387
мочь       0.009536
большой    0.008231
время      0.007394
цена       0.007252
тред       0.007022
работа     0.007022
блядь      0.006399
хохлов     0.005530
ещё        0.005447
нахуй      0.005286
дебил      0.005200
2          0.005126
хуй        0.005031


In [207]:
mnb = MultinomialNB(alpha=1.0)
mnb.fit(X, y)

preds_mnb = mnb.predict(X_test)

print(classification_report(y_test, preds_mnb, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.81      0.96      0.88       967
         1.0       0.87      0.53      0.66       475

    accuracy                           0.82      1442
   macro avg       0.84      0.75      0.77      1442
weighted avg       0.83      0.82      0.80      1442



In [208]:
log_probs = mnb.feature_log_prob_

features = vectorizer.get_feature_names_out()


importance_df = pd.DataFrame({
    'feature': features,
    'importance_class_0': log_probs[0],
    'importance_class_1': log_probs[1],
})


importance_df['importance'] = np.abs(importance_df['importance_class_0'])

importance_df = importance_df.sort_values(by='importance', ascending=False)

print(importance_df[['feature', 'importance']].head(10))

         feature  importance
4261    руснявый   10.338406
947       гитлер   10.338406
2471     москаль   10.338406
5588       шария   10.338406
3149    отсосать   10.338406
2507      мразь,   10.338406
1626    зашквара   10.338406
4348     сделал,   10.338406
5135         ты.   10.338406
5593  шизофреник   10.338406


In [209]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5000, class_weight='balanced')
rf.fit(X, y)

preds_rf = rf.predict(X_test)

print(classification_report(y_test, preds_rf, zero_division=0))


              precision    recall  f1-score   support

         0.0       0.85      0.83      0.84       967
         1.0       0.67      0.70      0.68       475

    accuracy                           0.79      1442
   macro avg       0.76      0.76      0.76      1442
weighted avg       0.79      0.79      0.79      1442



In [210]:
features_rf = vectorizer.get_feature_names_out()

feat_importances = pd.DataFrame(
    rf.feature_importances_,
    index=features_rf,
    columns=["Importance"]
)

feat_importances.sort_values(by="Importance", ascending=False, inplace=True)

print(feat_importances.head(20))

          Importance
год         0.010020
хохол       0.009929
очень       0.009200
русский     0.008175
свой        0.007901
твой        0.007334
тупой       0.006588
время       0.005550
мочь        0.005363
тред        0.005118
цена        0.004723
нахуй       0.004371
работа      0.004330
хуй         0.004277
хохлов      0.004219
купить      0.003829
работать    0.003753
большой     0.003610
город       0.003446
блядь       0.003422
